# Impor packages

In [21]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [22]:
import os
os.getcwd()

'C:\\Users\\USER\\Project_Python\\MySkill\\Market Basket Analysis_Online Retail Data'

# Memuat Dataset

In [23]:
df = pd.read_csv('Online Retail Data.csv', header=0)
df

,order_id,product_code,product_name,quantity,order_date,price,customer_id
0,493410,TEST001,This is a test product.,5,2010-01-04 09:24:00,4.50,12346.0
1,C493411,21539,RETRO SPOTS BUTTER DISH,-1,2010-01-04 09:43:00,4.25,14590.0
2,493412,TEST001,This is a test product.,5,2010-01-04 09:53:00,4.50,12346.0
3,493413,21724,PANDA AND BUNNIES STICKER SHEET,1,2010-01-04 09:54:00,0.85,NaN
4,493413,84578,ELEPHANT TOY WITH BLUE T-SHIRT,1,2010-01-04 09:54:00,3.75,NaN
...,...,...,...,...,...,...,...
461768,539991,21618,4 WILDFLOWER BOTANICAL CANDLES,1,2010-12-23 16:49:00,1.25,NaN
461769,539991,72741,GRAND CHOCOLATECANDLE,4,2010-12-23 16:49:00,1.45,NaN
461770,539992,21470,FLOWER VINE RAFFIA FOOD COVER,1,2010-12-23 17:41:00,3.75,NaN
461771,539992,22258,FELT FARM ANIMAL RABBIT,1,2010-12-23 17:41:00,1.25,NaN


## Assessing Data

**Memeriksa tipe data**

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461773 entries, 0 to 461772
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      461773 non-null  object 
 1   product_code  461773 non-null  object 
 2   product_name  459055 non-null  object 
 3   quantity      461773 non-null  int64  
 4   order_date    461773 non-null  object 
 5   price         461773 non-null  float64
 6   customer_id   360853 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 24.7+ MB


### Insight :
- **Missing Values**: Ada nilai yang hilang dalam kolom `product_name` dan `customer_id`. Ini perlu ditangani dalam langkah pembersihan data.
- **Tipe Data**: Kolom `order_date` sebaiknya dikonversi ke tipe datetime untuk analisis lebih lanjut yang melibatkan tanggal.
- **Jumlah Data**: Dengan lebih dari 460 ribu entri, data ini cukup besar dan mungkin memerlukan teknik khusus untuk pemrosesan efisien.

**Menghitung jumlah nilai unik di setiap kolom**

In [25]:
df.nunique()

order_id        25301
product_code     4508
product_name     4491
quantity          737
order_date      22368
price            1448
customer_id      3940
dtype: int64

**Memeriksa duplikasi data**

In [26]:
# Menghitung & Menampilkan jumlah baris dengan duplikasi values
print("Jumlah duplikasi: ", df.duplicated().sum())

Jumlah duplikasi:  6479


In [27]:
from fast_ml.feature_selection import get_duplicate_features

# Mendapatkan kolom-kolom dengan duplikasi values
duplicate_features = get_duplicate_features(df)
print('Duplikasi kolom:\n')
print(duplicate_features)

Duplikasi kolom:

Empty DataFrame
Columns: [Desc, feature1, feature2]
Index: []


**Memeriksa missing values**

In [28]:
print(df.isnull().sum())

order_id             0
product_code         0
product_name      2718
quantity             0
order_date           0
price                0
customer_id     100920
dtype: int64


**Memeriksa parameter statistik dari kolom numerik**

In [29]:
df.describe()

,quantity,price,customer_id
count,461773.000000,461773.000000,360853.000000
mean,9.088892,4.569963,15557.611138
std,86.366467,153.411566,1593.728741
min,-9600.000000,-53594.360000,12346.000000
25%,1.000000,1.250000,14210.000000
50%,3.000000,2.100000,15580.000000
75%,10.000000,4.210000,16938.000000
max,10200.000000,25111.090000,18287.000000


### Insight :
- **Outliers dan Data Kotor**: Terdapat nilai negatif dalam kolom `quantity` dan `price` yang mungkin perlu ditinjau dan diperbaiki atau dihapus.
- **Distribusi Data**: Terdapat variasi yang besar dalam data, baik dalam jumlah barang yang dipesan maupun harga barang.
- **Missing Values**: Sejumlah besar baris tidak memiliki `customer_id`, yang dapat mempengaruhi analisis.

Analisis lebih lanjut dapat dilakukan untuk **menangani outliers, missing values, dan mengidentifikasi pola-pola penting dalam data.**

# Data Cleansing

**Menangani missing value**

In [30]:
df[df.customer_id.isna()].head()

,order_id,product_code,product_name,quantity,order_date,price,customer_id
3,493413,21724,PANDA AND BUNNIES STICKER SHEET,1,2010-01-04 09:54:00,0.85,NaN
4,493413,84578,ELEPHANT TOY WITH BLUE T-SHIRT,1,2010-01-04 09:54:00,3.75,NaN
5,493413,21723,ALPHABET HEARTS STICKER SHEET,1,2010-01-04 09:54:00,0.85,NaN
477,493485,72008,FROSTED BOX 9 WHITE T-LIGHT CANDLES,24,2010-01-04 14:48:00,0.42,NaN
478,493485,21589,SWALLOW GIANT TUBE MATCHES,2,2010-01-04 14:48:00,2.55,NaN


In [31]:
# menghapus semua baris tanpa customer_id
df = df[~df['customer_id'].isna()]

In [32]:
df[df.product_name.isna()].head()

,order_id,product_code,product_name,quantity,order_date,price,customer_id


In [33]:
print(df.isnull().sum())

order_id        0
product_code    0
product_name    0
quantity        0
order_date      0
price           0
customer_id     0
dtype: int64


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 360853 entries, 0 to 461744
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      360853 non-null  object 
 1   product_code  360853 non-null  object 
 2   product_name  360853 non-null  object 
 3   quantity      360853 non-null  int64  
 4   order_date    360853 non-null  object 
 5   price         360853 non-null  float64
 6   customer_id   360853 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 22.0+ MB


**Menangani value pada kolom product_code dan product_name yang mengandung kata "test"**

In [35]:
# Memfilter baris yang product_code atau product_name mengandung kata 'test'
df_filtered = df[(df['product_code'].str.contains('test', case=False, na=False)) | (df['product_name'].str.contains('test', case=False, na=False))]

# Menghitung jumlah baris
jumlah_baris = len(df_filtered)

# Menampilkan jumlah baris
print("Jumlah baris yang yang product_code atau product_name mengandung kata 'test' :", jumlah_baris)

# Menampilkan hasil
df_filtered

Jumlah baris yang yang product_code atau product_name mengandung kata 'test' : 11


,order_id,product_code,product_name,quantity,order_date,price,customer_id
0,493410,TEST001,This is a test product.,5,2010-01-04 09:24:00,4.5,12346.0
2,493412,TEST001,This is a test product.,5,2010-01-04 09:53:00,4.5,12346.0
10146,494450,TEST001,This is a test product.,5,2010-01-14 13:50:00,4.5,12346.0
19402,495295,TEST001,This is a test product.,5,2010-01-22 13:30:00,4.5,12346.0
40332,497819,TEST001,This is a test product.,5,2010-02-12 14:58:00,0.0,14103.0
40427,497843,TEST001,This is a test product.,5,2010-02-12 15:47:00,0.0,14827.0
102946,504188,TEST001,This is a test product.,5,2010-04-12 09:52:00,4.5,16454.0
300544,C525272,TEST001,This is a test product.,-1,2010-10-04 16:29:00,4.5,16454.0
300545,C525273,TEST001,This is a test product.,-1,2010-10-04 16:30:00,4.5,16454.0
300546,C525274,TEST001,This is a test product.,-1,2010-10-04 16:33:00,4.5,12346.0


In [36]:
# menghapus semua baris dengan product_code atau product_name test
df = df[(~df['product_code'].str.lower().str.contains('test')) |
        (~df['product_name'].str.contains('test '))]

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 360842 entries, 1 to 461744
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      360842 non-null  object 
 1   product_code  360842 non-null  object 
 2   product_name  360842 non-null  object 
 3   quantity      360842 non-null  int64  
 4   order_date    360842 non-null  object 
 5   price         360842 non-null  float64
 6   customer_id   360842 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 22.0+ MB


**Menangani duplikasi data**

In [38]:
print("Jumlah duplikasi: ", df.duplicated().sum())

# Mengidentifikasi baris yang memiliki duplikasi berdasarkan semua kolom
duplikat = df[df.duplicated(keep=False)]

# Menampilkan baris yang memiliki duplikasi
print("Baris yang memiliki duplikasi:")
duplikat

Jumlah duplikasi:  6411
Baris yang memiliki duplikasi:


,order_id,product_code,product_name,quantity,order_date,price,customer_id
160,493435,21678,PAISLEY PATTERN STICKERS,2,2010-01-04 12:57:00,0.85,13206.0
173,493435,21678,PAISLEY PATTERN STICKERS,2,2010-01-04 12:57:00,0.85,13206.0
254,493442,20751,FUNKY WASHING UP GLOVES ASSORTED,1,2010-01-04 13:36:00,2.10,13821.0
255,493442,20751,FUNKY WASHING UP GLOVES ASSORTED,1,2010-01-04 13:36:00,2.10,13821.0
264,493442,20712,JUMBO BAG WOODLAND ANIMALS,1,2010-01-04 13:36:00,1.95,13821.0
...,...,...,...,...,...,...,...
459129,539653,22834,HAND WARMER BABUSHKA DESIGN,24,2010-12-20 16:50:00,0.85,13564.0
459227,539659,22728,ALARM CLOCK BAKELIKE PINK,6,2010-12-21 09:47:00,3.75,15034.0
459228,539659,22728,ALARM CLOCK BAKELIKE PINK,6,2010-12-21 09:47:00,3.75,15034.0
461725,539988,20751,FUNKY WASHING UP GLOVES ASSORTED,2,2010-12-23 16:06:00,2.10,18116.0


In [39]:
df.drop_duplicates(inplace=True)

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 354431 entries, 1 to 461744
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      354431 non-null  object 
 1   product_code  354431 non-null  object 
 2   product_name  354431 non-null  object 
 3   quantity      354431 non-null  int64  
 4   order_date    354431 non-null  object 
 5   price         354431 non-null  float64
 6   customer_id   354431 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 21.6+ MB


**Menangani Inaccurate value**

In [41]:
# mengubah nilai quantity yang negatif menjadi positif karena nilai negatif tersebut hanya menandakan order tersebut cancelled
df['quantity'] = df['quantity'].abs()

In [42]:
# membuat semua product_name berhuruf kecil
df['product_name'] = df['product_name'].str.lower()

In [43]:
# Memfilter baris dengan price bernilai negatif
df_negatif_price = df[df['price'] < 0]

# Menghitung jumlah baris dengan price bernilai negatif
jumlah_baris_negatif = len(df_negatif_price)

# Menampilkan jumlah baris
print("Jumlah baris dengan price bernilai negatif:", jumlah_baris_negatif)

# Menampilkan beberapa baris pertama dengan price bernilai negatif
print(df_negatif_price.head())

Jumlah baris dengan price bernilai negatif: 0
Empty DataFrame
Columns: [order_id, product_code, product_name, quantity, order_date, price, customer_id]
Index: []


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 354431 entries, 1 to 461744
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      354431 non-null  object 
 1   product_code  354431 non-null  object 
 2   product_name  354431 non-null  object 
 3   quantity      354431 non-null  int64  
 4   order_date    354431 non-null  object 
 5   price         354431 non-null  float64
 6   customer_id   354431 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 21.6+ MB


**Membuat kolom baru**

In [45]:
# membuat kolom amount, yaitu perkalian antara quantity dan price
df['amount'] = df['quantity'] * df['price']

In [46]:
# membuat kolom date
df['date'] = pd.to_datetime(df['order_date']).dt.date.astype('datetime64[ns]')

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 354431 entries, 1 to 461744
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      354431 non-null  object        
 1   product_code  354431 non-null  object        
 2   product_name  354431 non-null  object        
 3   quantity      354431 non-null  int64         
 4   order_date    354431 non-null  object        
 5   price         354431 non-null  float64       
 6   customer_id   354431 non-null  float64       
 7   amount        354431 non-null  float64       
 8   date          354431 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 27.0+ MB


**Mengganti product_name dari product_code yang memiliki beberapa product_name dengan salah satu product_name-nya yang paling sering muncul**

Langkah ini perlu dilakukan dalam analisa karena beberapa alasan berikut:

### Konsistensi dan Keakuratan Data:
1. **Mengatasi Ambiguitas**:
   - Kode produk (`product_code`) yang sama mungkin memiliki beberapa nama produk (`product_name`) yang berbeda di dalam dataset. Ini bisa terjadi karena kesalahan input data, variasi penamaan, atau pembaruan produk. Menggunakan satu nama produk yang paling sering muncul untuk setiap kode produk membantu mengatasi ambiguitas ini dan memastikan konsistensi data.

2. **Mengeliminasi Kebingungan**:
   - Dengan memilih nama produk yang paling sering muncul, kita mengeliminasi kebingungan yang mungkin timbul akibat adanya beberapa nama produk untuk satu kode produk yang sama. Hal ini membuat analisis data menjadi lebih sederhana dan dapat diandalkan.

### Mempermudah Analisis dan Visualisasi:
1. **Ringkas dan Jelas**:
   - Dataset yang lebih ringkas dengan nama produk yang seragam untuk setiap kode produk akan mempermudah analisis lebih lanjut dan visualisasi data. Ini membantu dalam memahami tren dan pola dengan lebih cepat dan jelas.

2. **Efektivitas Penjualan**:
   - Mengetahui produk mana yang paling sering dipesan di antara berbagai nama produk yang ada membantu dalam menilai efektivitas dan popularitas produk. Hal ini merupakan bagian penting dari analisis segmentasi pengguna, yang bertujuan untuk memahami perilaku pengguna dan preferensi mereka.

### Peningkatan Model Prediksi:
1. **Model Prediktif yang Lebih Baik**:
   - Dengan data yang lebih konsisten, model prediksi dapat dilatih dengan lebih baik dan memberikan hasil yang lebih akurat. Variasi dalam penamaan produk dapat menjadi gangguan dalam proses pelatihan model machine learning, sehingga dengan mengurangi variasi ini, kita mendapatkan model yang lebih kuat.

2. **Analisis Tren yang Akurat**:
   - Menggunakan nama produk yang konsisten untuk setiap kode produk membantu dalam melakukan analisis tren yang lebih akurat. Anda dapat mengidentifikasi tren popularitas produk dari waktu ke waktu dengan lebih tepat.

In [48]:
# mengelompokkan DataFrame df berdasarkan kombinasi product_code dan product_name, 
# kemudian menghitung jumlah unik order_id untuk setiap kombinasi, 
# dan akhirnya mengurutkan hasil berdasarkan product_code dan jumlah pesanan (order_cnt).
most_freq_product_name = df.groupby(['product_code','product_name'], as_index=False).agg(order_cnt=('order_id','nunique')).sort_values(['product_code','order_cnt'], ascending=[True,False])
most_freq_product_name

,product_code,product_name,order_cnt
0,10002,inflatable political globe,204
1,10080,groovy cactus inflatable,3
2,10120,doggy rubber,31
3,10123C,hearts wrapping tape,32
4,10123G,army camo wrapping tape,11
...,...,...,...
4304,D,discount,78
4305,M,manual,482
4306,PADS,pads to match all cushions,14
4307,POST,postage,57


In [49]:
# mengelompokkan DataFrame most_freq_product_name berdasarkan product_code dan memilih kolom order_cnt untuk perhitungan selanjutnya.
# memberi peringkat pada nilai dalam kolom order_cnt untuk setiap kelompok product_code.
# Hasil dari perhitungan peringkat disimpan dalam kolom baru bernama rank di DataFrame most_freq_product_name.
most_freq_product_name['rank'] = most_freq_product_name.groupby('product_code')['order_cnt'].rank(method='first', ascending=False)
most_freq_product_name

,product_code,product_name,order_cnt,rank
0,10002,inflatable political globe,204,1.0
1,10080,groovy cactus inflatable,3,1.0
2,10120,doggy rubber,31,1.0
3,10123C,hearts wrapping tape,32,1.0
4,10123G,army camo wrapping tape,11,1.0
...,...,...,...,...
4304,D,discount,78,1.0
4305,M,manual,482,1.0
4306,PADS,pads to match all cushions,14,1.0
4307,POST,postage,57,1.0


Tujuan keseluruhan dari langkah ini adalah untuk mengidentifikasi produk mana yang paling sering dipesan (memiliki `order_cnt` tertinggi) untuk setiap `product_code`. Dengan informasi peringkat ini, kita dapat:
- Memilih produk dengan jumlah pesanan terbanyak untuk setiap `product_code`.
- Menentukan produk mana yang paling populer atau paling laris dalam setiap kategori kode produk.

In [50]:
# memfilter DataFrame most_freq_product_name sehingga hanya menyertakan baris di mana nilai rank adalah 1
# kemudian menghapus kolom order_cnt dan rank dari DataFrame. 
# Kolom-kolom ini tidak lagi diperlukan setelah kita mendapatkan produk dengan nama yang paling sering muncul untuk setiap kode produk.
most_freq_product_name = most_freq_product_name[most_freq_product_name['rank']==1].drop(columns=['order_cnt','rank'])
most_freq_product_name

,product_code,product_name
0,10002,inflatable political globe
1,10080,groovy cactus inflatable
2,10120,doggy rubber
3,10123C,hearts wrapping tape
4,10123G,army camo wrapping tape
...,...,...
4304,D,discount
4305,M,manual
4306,PADS,pads to match all cushions
4307,POST,postage


In [51]:
# menggabungkan DataFrame asli df dengan DataFrame most_freq_product_name berdasarkan kolom product_code
# yang menunjukkan nama produk yang paling sering dipesan untuk setiap product_code
# Hal ini membantu dalam menyamakan nama produk untuk kode produk yang mungkin memiliki beberapa nama produk.
df = df.merge(most_freq_product_name.rename(columns={'product_name':'most_freq_product_name'}), how='left', on='product_code')
df

,order_id,product_code,product_name,quantity,order_date,price,customer_id,amount,date,most_freq_product_name
0,C493411,21539,retro spots butter dish,1,2010-01-04 09:43:00,4.25,14590.0,4.25,2010-01-04,red retrospot butter dish
1,493414,21844,retro spot mug,36,2010-01-04 10:28:00,2.55,14590.0,91.80,2010-01-04,red retrospot mug
2,493414,21533,retro spot large milk jug,12,2010-01-04 10:28:00,4.25,14590.0,51.00,2010-01-04,retro spot large milk jug
3,493414,37508,new england ceramic cake server,2,2010-01-04 10:28:00,2.55,14590.0,5.10,2010-01-04,new england ceramic cake server
4,493414,35001G,hand open shape gold,2,2010-01-04 10:28:00,4.25,14590.0,8.50,2010-01-04,hand open shape gold
...,...,...,...,...,...,...,...,...,...,...
354426,539988,84380,set of 3 butterfly cookie cutters,1,2010-12-23 16:06:00,1.25,18116.0,1.25,2010-12-23,set of 3 butterfly cookie cutters
354427,539988,84849D,hot baths soap holder,1,2010-12-23 16:06:00,1.69,18116.0,1.69,2010-12-23,hot baths soap holder
354428,539988,84849B,fairy soap soap holder,1,2010-12-23 16:06:00,1.69,18116.0,1.69,2010-12-23,fairy soap soap holder
354429,539988,22854,cream sweetheart egg holder,2,2010-12-23 16:06:00,4.95,18116.0,9.90,2010-12-23,cream sweetheart egg holder


In [52]:
# mengganti nilai dalam kolom product_name dengan nilai dalam kolom most_freq_product_name 
# dan kemudian menghapus kolom most_freq_product_name dari DataFrame df
df['product_name'] = df['most_freq_product_name']
df = df.drop(columns='most_freq_product_name')
df

,order_id,product_code,product_name,quantity,order_date,price,customer_id,amount,date
0,C493411,21539,red retrospot butter dish,1,2010-01-04 09:43:00,4.25,14590.0,4.25,2010-01-04
1,493414,21844,red retrospot mug,36,2010-01-04 10:28:00,2.55,14590.0,91.80,2010-01-04
2,493414,21533,retro spot large milk jug,12,2010-01-04 10:28:00,4.25,14590.0,51.00,2010-01-04
3,493414,37508,new england ceramic cake server,2,2010-01-04 10:28:00,2.55,14590.0,5.10,2010-01-04
4,493414,35001G,hand open shape gold,2,2010-01-04 10:28:00,4.25,14590.0,8.50,2010-01-04
...,...,...,...,...,...,...,...,...,...
354426,539988,84380,set of 3 butterfly cookie cutters,1,2010-12-23 16:06:00,1.25,18116.0,1.25,2010-12-23
354427,539988,84849D,hot baths soap holder,1,2010-12-23 16:06:00,1.69,18116.0,1.69,2010-12-23
354428,539988,84849B,fairy soap soap holder,1,2010-12-23 16:06:00,1.69,18116.0,1.69,2010-12-23
354429,539988,22854,cream sweetheart egg holder,2,2010-12-23 16:06:00,4.95,18116.0,9.90,2010-12-23


Hasilnya adalah DataFrame `df` di mana kolom `product_name` berisi nama produk yang paling sering dipesan untuk setiap `product_code`, dan kolom `most_freq_product_name` telah dihapus untuk menjaga kebersihan dan keterbacaan data.

Langkah ini membantu dalam konsolidasi dan penyederhanaan data produk untuk analisis retensi pengguna. 

In [53]:
# mengkonversi customer_id menjadi string
df['customer_id'] = df['customer_id'].astype(str)

Perintah ini digunakan untuk mengubah tipe data kolom `customer_id` menjadi string. 

### Mengapa Ini Penting?
- **Konsistensi Data**: Mengubah `customer_id` menjadi string memastikan bahwa semua nilai dalam kolom tersebut diperlakukan sebagai teks. Ini berguna jika  ingin menghindari perhitungan matematis atau operasi numerik yang tidak disengaja.
- **Pemrosesan Data**: Banyak operasi pemrosesan data lebih mudah dilakukan dengan string, terutama jika `customer_id` berisi karakter non-numerik atau panjang yang bervariasi.
- **Pengenalan Unik**: ID pelanggan biasanya bersifat unik dan lebih tepat jika diperlakukan sebagai string, sehingga tidak ada penafsiran yang salah sebagai nilai numerik.

Langkah ini adalah bagian dari proses pembersihan data, memastikan bahwa semua data dalam kolom `customer_id` sesuai dengan tipe yang diharapkan untuk analisis lebih lanjut.

In [54]:
# menghapus outlier
from scipy import stats
df = df[(np.abs(stats.zscore(df[['quantity','amount']]))<3).all(axis=1)]
df = df.reset_index(drop=True)
df

,order_id,product_code,product_name,quantity,order_date,price,customer_id,amount,date
0,C493411,21539,red retrospot butter dish,1,2010-01-04 09:43:00,4.25,14590.0,4.25,2010-01-04
1,493414,21844,red retrospot mug,36,2010-01-04 10:28:00,2.55,14590.0,91.80,2010-01-04
2,493414,21533,retro spot large milk jug,12,2010-01-04 10:28:00,4.25,14590.0,51.00,2010-01-04
3,493414,37508,new england ceramic cake server,2,2010-01-04 10:28:00,2.55,14590.0,5.10,2010-01-04
4,493414,35001G,hand open shape gold,2,2010-01-04 10:28:00,4.25,14590.0,8.50,2010-01-04
...,...,...,...,...,...,...,...,...,...
352074,539988,84380,set of 3 butterfly cookie cutters,1,2010-12-23 16:06:00,1.25,18116.0,1.25,2010-12-23
352075,539988,84849D,hot baths soap holder,1,2010-12-23 16:06:00,1.69,18116.0,1.69,2010-12-23
352076,539988,84849B,fairy soap soap holder,1,2010-12-23 16:06:00,1.69,18116.0,1.69,2010-12-23
352077,539988,22854,cream sweetheart egg holder,2,2010-12-23 16:06:00,4.95,18116.0,9.90,2010-12-23


Jika langkah menghapus outlier ini tidak dilakukan dalam analisis, beberapa masalah dapat muncul:

1. **Distorsi Hasil Analisis**: Outlier dapat sangat mempengaruhi perhitungan statistik seperti mean, median, dan standar deviasi. Ini dapat menyebabkan hasil analisis yang tidak akurat atau menyesatkan.

2. **Kesalahan Model Prediksi**: Dalam konteks machine learning, model yang dilatih dengan data yang mengandung outlier mungkin belajar pola yang tidak representatif dari populasi data yang sebenarnya. Ini dapat mengurangi akurasi model prediksi dan membuatnya kurang dapat diandalkan.

3. **Kesimpulan yang Salah**: Analisis yang mengandalkan data yang mengandung outlier mungkin menghasilkan kesimpulan yang salah. Misalnya, kita mungkin mengidentifikasi tren atau pola yang sebenarnya tidak ada karena adanya nilai-nilai ekstrem tersebut.

4. **Keputusan Bisnis yang Buruk**: Hasil analisis yang terdistorsi dapat menyebabkan pengambilan keputusan bisnis yang buruk. Sebagai contoh, kita mungkin mengalokasikan sumber daya atau merumuskan strategi pemasaran berdasarkan data yang tidak akurat.

5. **Mengabaikan Data Penting**: Dalam beberapa kasus, outlier sebenarnya dapat menunjukkan kejadian penting atau pola yang perlu diteliti lebih lanjut. Namun, jika tidak disaring dan dianalisis dengan tepat, informasi ini mungkin akan terlewatkan atau diabaikan.

Dengan demikian, menghapus outlier adalah langkah penting dalam memastikan kualitas dan keandalan analisis retensi pengguna. Ini membantu untuk mendapatkan gambaran yang lebih akurat tentang perilaku dan tren pelanggan, yang pada akhirnya akan mendukung pengambilan keputusan yang lebih baik.

In [55]:
# menghapus baris dengan status cancelled, yaitu yang order_id-nya diawali 'C'
df = df[df['order_id'].str[:1]!='C']

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 344344 entries, 1 to 352078
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      344344 non-null  object        
 1   product_code  344344 non-null  object        
 2   product_name  344344 non-null  object        
 3   quantity      344344 non-null  int64         
 4   order_date    344344 non-null  object        
 5   price         344344 non-null  float64       
 6   customer_id   344344 non-null  object        
 7   amount        344344 non-null  float64       
 8   date          344344 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 26.3+ MB


# Menyiapkan data basket

**Buat DataFrame basket**

In [57]:
basket = pd.pivot_table(df, index='order_id', columns='product_name', values='product_code', aggfunc='nunique', fill_value=0)
basket

product_name,10 colour spaceboy pen,12 ass zinc christmas decorations,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 message cards with envelopes,12 mini toadstool pegs,12 pencil small tube woodland,12 pencils small tube posy,...,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc police box lantern,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493414,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493427,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493428,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493432,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493433,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539981,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
539982,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
539985,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Pivot table basket akan menampilkan jumlah produk unik yang dipesan dalam setiap transaksi (order_id) untuk setiap product_name. Baris dari pivot table mewakili transaksi (order_id), dan kolom mewakili nama produk (product_name). Nilai dalam tabel adalah 0, ini menunjukkan bahwa tidak ada produk yang dipesan dalam transaksi tersebut. Sebaliknya, jika nilai adalah 1, maka produk tersebut dipesan dalam transaksi.

In [58]:
basket.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16356 entries, 493414 to 539988
Columns: 3846 entries, 10 colour spaceboy pen to zinc willie winkie  candle stick
dtypes: int64(3846)
memory usage: 480.1+ MB


**Encode DataFrame basket dengan nilai True untuk semua nilai di atas 0 dan False untuk semua nilai 0**

In [59]:
def encode(x):
    if x==0:
        return False
    if x>0:
        return True

basket_encode = basket.applymap(encode)
basket_encode

product_name,10 colour spaceboy pen,12 ass zinc christmas decorations,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 message cards with envelopes,12 mini toadstool pegs,12 pencil small tube woodland,12 pencils small tube posy,...,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc police box lantern,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493414,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493427,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493428,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493432,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493433,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539981,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539982,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539985,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


- Kode di atas digunakan untuk mengubah nilai dalam DataFrame basket menjadi nilai Boolean. 
- Fungsi ini mengonversi nilai-nilai dalam DataFrame basket:
    Jika nilai x adalah 0, maka akan dikembalikan False.
    Jika nilai x lebih dari 0, maka akan dikembalikan True.
- Hasilnya adalah DataFrame basket_encode di mana semua nilai 0 diubah menjadi False dan nilai lainnya diubah menjadi True.

In [60]:
basket_encode.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16356 entries, 493414 to 539988
Columns: 3846 entries, 10 colour spaceboy pen to zinc willie winkie  candle stick
dtypes: bool(3846)
memory usage: 60.1+ MB


**Ambil transaksi dengan banyaknya produk unik lebih dari 1 saja**

In [61]:
basket_filter = basket_encode[(basket_encode>0).sum(axis=1)>1]
basket_filter

product_name,10 colour spaceboy pen,12 ass zinc christmas decorations,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 message cards with envelopes,12 mini toadstool pegs,12 pencil small tube woodland,12 pencils small tube posy,...,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc police box lantern,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493414,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493427,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493428,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493432,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493433,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539978,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539981,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539982,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


- digunakan untuk memfilter transaksi yang memiliki lebih dari satu produk unik.
- output dari basket_filter akan berisi transaksi di mana pelanggan membeli lebih dari satu produk unik. Data ini berguna untuk analisis lebih lanjut seperti mencari pola pembelian, analisis keranjang pasar, atau rekomendasi produk.

In [62]:
basket_filter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15062 entries, 493414 to 539988
Columns: 3846 entries, 10 colour spaceboy pen to zinc willie winkie  candle stick
dtypes: bool(3846)
memory usage: 55.4+ MB


# Mengaplikasikan apriori algorithm

**Buat list frequent itemset (kumpulan produk yang sering dibeli)**

In [44]:
# pip install mlxtend
from mlxtend.frequent_patterns import apriori

frequent_itemset = apriori(basket_filter, min_support=.01, use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)
frequent_itemset['product_cnt'] = frequent_itemset['itemsets'].apply(lambda x: len(x))
frequent_itemset

MemoryError: Unable to allocate 6.55 GiB for an array with shape (233586, 2, 15062) and data type bool

Masalah ini terjadi karena ukuran data yang terlalu besar untuk dimuat ke dalam memori saat menjalankan algoritma Apriori. Untuk mengatasi ini, kita dapat mencoba beberapa pendekatan berikut:
- Mengurangi Dimensi Data: Hanya gunakan subset dari data yang lebih kecil untuk analisis awal.
- Menyesuaikan Parameter Algoritma Apriori: Ubah parameter min_support untuk meningkatkan threshold minimum support sehingga lebih sedikit itemset yang diproses.

In [85]:
from mlxtend.frequent_patterns import apriori

# Menggunakan hanya sebagian data untuk mengurangi ukuran memori yang diperlukan
basket_sample = basket_filter.sample(frac=0.1, random_state=42)

frequent_itemset = apriori(basket_sample, min_support=.01, use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)
frequent_itemset['product_cnt'] = frequent_itemset['itemsets'].apply(lambda x: len(x))
frequent_itemset

,support,itemsets,product_cnt
0,0.179947,(white hanging heart t-light holder),1
1,0.095618,(jumbo bag red retrospot),1
2,0.094290,(regency cakestand 3 tier),1
3,0.087649,(pack of 72 retro spot cake cases),1
4,0.080345,(lunch bag red spotty),1
...,...,...,...
1296,0.010624,"(jumbo bag baroque black white, jumbo shopper...",3
1297,0.010624,"(lunch bag spaceboy design, childrens apron sp...",2
1298,0.010624,(red spotty apron),1
1299,0.010624,"(childs garden brush blue, childs garden brush...",2


**Hitung nilai support, confidence, dan lift dari setiap pasangan produk yang mungkin**

In [88]:
from mlxtend.frequent_patterns import association_rules

product_association = association_rules(frequent_itemset, metric='confidence', min_threshold=.7).sort_values(['support','confidence'], ascending=[False,False]).reset_index(drop=True)
product_association

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(red hanging heart t-light holder),(white hanging heart t-light holder),0.052457,0.179947,0.040505,0.772152,4.290999,1.0,0.031065,3.599122,0.809413,0.211073,0.722154,0.498622
1,(sweetheart ceramic trinket box),(strawberry ceramic trinket box),0.041833,0.062417,0.031873,0.761905,12.206687,1.0,0.029261,3.937849,0.958160,0.440367,0.746054,0.636272
2,(jumbo bag spaceboy design),(jumbo bag pink with white spots),0.041169,0.071049,0.029880,0.725806,10.215556,1.0,0.026955,3.387938,0.940843,0.362903,0.704835,0.573184
3,(pink blue felt craft trinket box),(pink cream felt craft trinket box),0.033865,0.049801,0.024568,0.725490,14.567843,1.0,0.022882,3.461440,0.964001,0.415730,0.711103,0.609412
4,(strawberry charlotte bag),(red spotty charlotte bag),0.029216,0.043825,0.023240,0.795455,18.150826,1.0,0.021960,4.674635,0.973344,0.466667,0.786080,0.662879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,"(72 sweetheart fairy cake cases, pack of 60 di...",(60 teatime fairy cake cases),0.014608,0.057769,0.010624,0.727273,12.589342,1.0,0.009780,3.454847,0.934215,0.172043,0.710552,0.455590
200,"(72 sweetheart fairy cake cases, pack of 60 di...",(pack of 60 pink paisley cake cases),0.014608,0.057105,0.010624,0.727273,12.735729,1.0,0.009790,3.457282,0.935142,0.173913,0.710755,0.456660
201,(lush greens ribbons),(bright blues ribbons),0.014608,0.018592,0.010624,0.727273,39.116883,1.0,0.010353,3.598495,0.988881,0.470588,0.722106,0.649351
202,"(jumbo bag toys, jumbo storage bag suki)",(jumbo bag pink with white spots),0.014608,0.071049,0.010624,0.727273,10.236194,1.0,0.009586,3.406153,0.915684,0.141593,0.706414,0.438403


## Insight :

### 1. **Hubungan Produk dengan Confidence dan Lift Tinggi**
- **`red hanging heart t-light holder -> white hanging heart t-light holder`** memiliki confidence sebesar 77.22% dan lift 4.29. Ini menunjukkan bahwa pelanggan yang membeli `red hanging heart t-light holder` sangat mungkin untuk juga membeli `white hanging heart t-light holder`.
  
- **`sweetheart ceramic trinket box -> strawberry ceramic trinket box`** memiliki confidence 76.19% dan lift 12.21. Produk ini sering dibeli bersama, menunjukkan hubungan yang kuat antara kedua produk ini.

### 2. **Produk yang Sering Dibeli Bersama**
- **`jumbo bag spaceboy design -> jumbo bag pink with white spots`** dengan confidence 72.58% dan lift 10.22. Ini menunjukkan bahwa pelanggan yang membeli `jumbo bag spaceboy design` juga sering membeli `jumbo bag pink with white spots`.

- **`pink blue felt craft trinket box -> pink cream felt craft trinket box`** memiliki confidence 72.55% dan lift 14.57. Produk-produk kerajinan ini memiliki keterkaitan yang kuat, menunjukkan bahwa mereka mungkin menarik bagi segmen pelanggan yang sama.

### 3. **Peluang untuk Bundling Produk**
- Produk seperti **`strawberry charlotte bag -> red spotty charlotte bag`** dengan confidence 79.55% dan lift 18.15 menunjukkan bahwa ada potensi besar untuk membuat bundling produk ini untuk meningkatkan penjualan.

### 4. **Rekomendasi Produk Berdasarkan Pola Pembelian**
- Berdasarkan aturan asosiasi ini, Anda bisa membuat rekomendasi produk yang lebih personal kepada pelanggan. Misalnya, jika pelanggan membeli produk `sweetheart ceramic trinket box`, Anda bisa merekomendasikan mereka untuk juga membeli `strawberry ceramic trinket box`.

### 5. **Analisis Produk Berbeda untuk Strategi Pemasaran**
- Untuk produk seperti **`jumbo storage bag suki -> jumbo shopper vintage red paisley`** dengan confidence 72.73% dan lift 11.65, strategi pemasaran yang berfokus pada hubungan antara produk-produk ini bisa membantu meningkatkan penjualan.

### 6. **Tren Produk dan Preferensi Pelanggan**
- Produk yang memiliki lift tinggi seperti **`lush greens ribbons -> bright blues ribbons`** menunjukkan tren produk dan preferensi pelanggan yang kuat. Anda bisa menggunakan data ini untuk memahami lebih baik apa yang diminati pelanggan dan menyesuaikan penawaran produk Anda.